<a href="https://colab.research.google.com/github/mrj760/Data-Science-Assignments/blob/main/CS410_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from pathlib import Path
import pandas as pd
import matplotlib as mpl
import seaborn as sb
import plotly.express as pltx
from chardet import detect
import numpy as np
import os

##Restaurant Violations Dataset

In [ ]:
drive = Path() / '/content' / 'drive' / 'MyDrive' / 'CS410WranglingFiles'

In [ ]:
insp = pd.read_csv(drive / 'inspections.csv')
vio = pd.read_csv(drive / 'violations.csv')

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', 50)
pd.set_option("expand_frame_repr", False)

In [ ]:
print(insp)

       business_id  score      date     type
0               19     94  20160513  routine
1               19     94  20171211  routine
2               24     98  20171101  routine
3               24     98  20161005  routine
4               24     96  20160311  routine
...            ...    ...       ...      ...
14217        94012     90  20180112  routine
14218        94133    100  20171227  routine
14219        94142    100  20171220  routine
14220        94189     96  20171130  routine
14221        94231     85  20171214  routine

[14222 rows x 4 columns]


In [ ]:
print(vio)

       business_id      date                                        description
0               19  20171211  Inadequate food safety knowledge or lack of ce...
1               19  20171211   Unapproved or unmaintained equipment or utensils
2               19  20160513  Unapproved or unmaintained equipment or utensi...
3               19  20160513  Unclean or degraded floors walls or ceilings  ...
4               19  20160513  Food safety certificate or food handler card n...
...            ...       ...                                                ...
39037        94189  20171130            Insufficient hot water or running water
39038        94231  20171214  Unclean nonfood contact surfaces  [ date viola...
39039        94231  20171214  High risk vermin infestation  [ date violation...
39040        94231  20171214  Moderate risk food holding temperature   [ dat...
39041        94231  20171214  Wiping cloths not clean or properly stored or ...

[39042 rows x 3 columns]


In [ ]:
''' Determine the file format, file encoding, and file size 

inspections.csv : delimited by commas. 
Contains: numeric business id, a string score, a string date, and a string type
Granularity: Each row in the table represents an inspection of a business on a particular day

violations.csv : delimited by commas
Contains: numeric business id, a string date, and a string description
Granularity: Each row represents a violation of a business on a particular day '''

''' Getting inspections.csv file encoding and size '''
insp_bytes = detect((drive / 'inspections.csv').read_bytes()) # read in data frame as bytes for finding encoding
insp_size = os.path.getsize(drive / 'inspections.csv') # use os to find file size of dataframe 
print(f"inspections.csv [ encoding: {insp_bytes['encoding']}, confidence: {insp_bytes['confidence']}, file size: {int(insp_size / 1024)}KiB ]")

''' Getting violations.csv file encoding and size '''
vio_bytes = detect((drive / 'violations.csv').read_bytes()) # read in data frame as bytes for finding encoding
vio_size = os.path.getsize(drive / 'violations.csv') # use os to find file size of dataframe 
print(f"violations.csv  [ encoding: {vio_bytes['encoding']}, confidence: {vio_bytes['confidence']}, file size: {int(vio_size / 1024)}KiB ]")

inspections.csv [ encoding: ascii, confidence: 1.0, file size: 455KiB ]
violations.csv  [ encoding: ascii, confidence: 1.0, file size: 3638KiB ]


In [ ]:
''' take a date and use pandas to turn it into a date_time, then make a column in the dataframe overwrite the date column and add a year column '''
def parse_dates_and_years(df, column="date"):
  dates = pd.to_datetime(df[column], format="%Y%m%d")
  years = dates.dt.year
  return df.assign(date=dates, year=years)

''' apply the above function to the violations dataframe '''
vio = vio.pipe(parse_dates_and_years)

''' take whatever and return a 1 '''
def one(thing):
  return 1

''' remove everything in a string starting from the first '[' then also remove traling whitespace '''
def strip_date_correction(foo):
  idx = foo.find("[")
  if idx != -1:
    foo = foo[:idx]
  return foo.strip()

descs = vio['description'] # get descriptions column to work with

#################### Is this necessary? ####################
# vio = vio.assign(description=descs.apply(strip_date_correction)) # unify descriptions (remove text specificying date correction)
############################################################

In [ ]:
''' Focus on inspections that happened only in 2017. 
    Count the num. of violations in each category. 
    (Hint: use value_counts function in Pandas)
    Display the top 20 violations '''

''' make another col with just 1's '''
vio_counts_2017 = vio.assign(count=descs.apply(one)) 

''' get only the violations which occured in 2017 '''
vio_counts_2017 = vio_counts_2017.loc[vio_counts_2017['year'] == 2017] 

''' group by unified descriptions and sum the 1's for each group '''
vio_counts_2017 = vio_counts_2017.groupby('description')['count'].sum().reset_index() 

''' sort the rows by their new count value and print 20 rows with only relevant col.'s '''
print(vio_counts_2017.sort_values('count', ascending=False).reset_index().loc[:,['description', 'count']].head(20)) 

                                          description  count
0        Unclean or degraded floors walls or ceilings    743
1    Unapproved or unmaintained equipment or utensils    498
2                               Improper food storage    350
3   Inadequately cleaned or sanitized food contact...    341
4   Inadequate and inaccessible handwashing facili...    331
5   Wiping cloths not clean or properly stored or ...    306
6             Moderate risk food holding temperature     304
7                    Unclean nonfood contact surfaces    271
8                    Moderate risk vermin infestation    265
9              Foods not protected from contamination    255
10  Food safety certificate or food handler card n...    254
11                        Low risk vermin infestation    223
12     Permit license or inspection report not posted    208
13  Inadequate food safety knowledge or lack of ce...    192
14   Improper storage of equipment utensils or linens    184
15                     I

In [ ]:
''' Create a new feature num_vio to count the violations from every business '''

''' give all businesses a new column with a 1 '''
num_vio = vio['business_id']
bus_vios = vio.assign(num_vio=num_vio.apply(one))

''' turn dates into a more readable format and get the years so we can filter out inspections/violations from 2017 '''
insp = insp.pipe(parse_dates_and_years)
insp = insp.loc[insp['year'] == 2017]

''' sum up all the ones from earlier when grouping by both business id and year '''
bus_vios = bus_vios.groupby(['business_id', 'date'])['num_vio'].sum().reset_index()

''' merge the tables to get the score attribute with each date's violation count '''
bus_vios = bus_vios.merge(insp, on='business_id', how='right').rename(columns={'date_y': 'date'}).loc[:,['business_id','score','date','type','num_vio']].drop_duplicates('date')

print(bus_vios)

       business_id  score       date     type  num_vio
0               19     94 2017-12-11  routine      3.0
2               24     98 2017-11-01  routine      2.0
5               45     88 2017-03-07  routine      7.0
9               45     85 2017-09-14  routine      7.0
13              54     87 2017-02-15  routine      3.0
...            ...    ...        ...      ...      ...
12348        88840     96 2017-10-21  routine      3.0
12392        89114     86 2017-04-04  routine      5.0
12502        89678    100 2017-03-26  routine      NaN
12595        90137     98 2017-01-21  routine      1.0
12720        90792     96 2017-08-05  routine      1.0

[274 rows x 5 columns]


In [ ]:
''' Determine how many NaN values exist and explain how you will address this missing data.
    Deductive imputation? Mean imputation? Hot-deck imputation? Why?
    Write the code for your choice and display the result. '''
print(f"The number of null values in num_vio : {bus_vios['num_vio'].isnull().sum()}")

The number of null values in num_vio : 7


###I will use deductive imputation and say each NaN value should be replaced with 0 because those businesses don't have any violations on record

In [ ]:
''' Deductive imputation. Make all num_vio NaN values into 0 '''
bus_vios['num_vio'] = bus_vios['num_vio'].fillna(0)
print(bus_vios)

       business_id  score       date     type  num_vio
0               19     94 2017-12-11  routine      3.0
2               24     98 2017-11-01  routine      2.0
5               45     88 2017-03-07  routine      7.0
9               45     85 2017-09-14  routine      7.0
13              54     87 2017-02-15  routine      3.0
...            ...    ...        ...      ...      ...
12348        88840     96 2017-10-21  routine      3.0
12392        89114     86 2017-04-04  routine      5.0
12502        89678    100 2017-03-26  routine      0.0
12595        90137     98 2017-01-21  routine      1.0
12720        90792     96 2017-08-05  routine      1.0

[274 rows x 5 columns]
